In [2]:
import scanpy as sc
import pandas as pd
import anndata as ad
from anndata import AnnData

### Read Data

In [3]:
# Replace 'counts_matrix.txt' with the path to your file
counts = pd.read_csv("/scratch/sah2p/projects/G2PDeep-sc-LLM/data/GSE131907/GSE131907_raw_UMI_N_lung.txt", sep="\t", index_col=0)

In [4]:
adata = AnnData(X=counts.T.values)
adata.var_names = counts.index
adata.obs_names = counts.columns

In [5]:
# Split the cell barcodes to extract sample names and barcodes
adata.obs['barcode'] = adata.obs_names.str.split('_').str[0]  # Sample name
adata.obs['Tissue'] = adata.obs_names.str.split('_').str[1]
adata.obs['sample'] = adata.obs_names.str.split('_').str[2]  # Actual barcode

In [ ]:
#Seperate Anndata by Sample
# Create a list of AnnData objects split by sample
adata_by_sample = [adata[adata.obs['sample'] == sample].copy() for sample in adata.obs['sample'].unique()]
adata_by_sample


Additionally, it is important to note that for datasets with multiple batches, quality control should be performed for each sample individually as quality control thresholds can very substantially between batches.

One can now inspect violin plots of some of the computed QC metrics:

the number of genes expressed in the count matrix

the total counts per cell

the percentage of counts in mitochondrial genes

In [8]:
#Perform QC for each sample
for index, adata_sample in enumerate(adata_by_sample):
    # Calculate mitochondrial gene percentage
    adata_sample.var['mt'] = adata_sample.var_names.str.startswith('MT-')
     # ribosomal genes
    adata_sample.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
# hemoglobin genes
    adata_sample.var["hb"] = adata.var_names.str.contains("^HB[^(P)]")# Adjust prefix if necessary
    sc.pp.calculate_qc_metrics(adata_sample, qc_vars=['mt',"ribo", "hb"], inplace=True)
    
    # Filter cells based on QC metrics
    adata_sample = adata_sample[(adata_sample.obs['pct_counts_mt'] <= 20) &
                                (adata_sample.obs['n_genes_by_counts'] > 100) &
                                (adata_sample.obs['n_genes_by_counts'] < 150000) &
                                (adata_sample.obs['total_counts'] > 200) &
                                (adata_sample.obs['total_counts'] < 10000), :]
    
    remove = ['total_counts_mt', 'log1p_total_counts_mt', 'total_counts_ribo', 
          'log1p_total_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb']
    
    adata_sample.obs = adata_sample.obs[[x for x in adata_sample.obs.columns if x not in remove]]
    # Store the filtered AnnData object back in the dictionary
    adata_by_sample[index] = adata_sample
    

In [ ]:
sc.pp.scrublet(adata, batch_key="sample")


In [ ]:
for sample, adata_sample in enumerate(adata_by_sample):
    sc.pl.violin(
    adata_sample,
    ["n_genes_by_counts", "total_counts", "pct_counts_mt"],
    jitter=0.4,
    multi_panel=True,
)
    sc.pl.scatter(adata_sample, "total_counts", "n_genes_by_counts", color="pct_counts_mt")


Combining data for further analysis

In [10]:
adata_filtered = adata_by_sample[list(adata_by_sample.keys())[0]]

if len(adata_by_sample) > 1:
    cols_to_drop = ['n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts']
    adata_sample.obs.drop(cols_to_drop, axis=1, inplace=True, errors='ignore')

adata_filtered = adata_filtered.concatenate(adata_by_sample[sample].obs for sample in list(adata_by_sample.keys())[1:])

AttributeError: 'list' object has no attribute 'keys'

In [ ]:
adata_filtered

In [25]:
df = pd.concat([x.obs for x in adata_by_sample])
df.sort_values('sample')

,barcode,Tissue,sample,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes,pct_counts_mt,pct_counts_ribo,pct_counts_hb,doublet,doublet_score
ACTATCTAGACAGAGA_LUNG_N01,ACTATCTAGACAGAGA,LUNG,N01,1254,7.134891,3020,8.013343,36.754967,48.874172,60.033113,75.033113,5.794702,21.622517,0.000000,0.0,0.000015
CCGGGATCACCTCGGA_LUNG_N01,CCGGGATCACCTCGGA,LUNG,N01,986,6.894670,2406,7.786136,37.032419,50.083126,62.967581,79.800499,5.320033,25.768911,0.000000,0.0,0.055152
CCGGGATCAGCTCGAC_LUNG_N01,CCGGGATCAGCTCGAC,LUNG,N01,1326,7.190676,3362,8.120589,37.477692,50.297442,61.511005,75.431291,4.878049,26.650803,0.000000,0.0,3.322630
CCGGTAGAGCACAGGT_LUNG_N01,CCGGTAGAGCACAGGT,LUNG,N01,547,6.306275,1198,7.089243,42.988314,59.432387,71.035058,96.076795,7.262104,38.397329,0.000000,0.0,2.092028
CCGGTAGAGCTAGGCA_LUNG_N01,CCGGTAGAGCTAGGCA,LUNG,N01,1204,7.094235,2815,7.943073,34.103020,47.069272,59.005329,74.991119,3.161634,22.202487,0.000000,0.0,0.209435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CCCAGTTTCAACGGCC_LUNG_N34,CCCAGTTTCAACGGCC,LUNG,N34,1022,6.930495,2903,7.973844,39.614192,53.806407,67.378574,82.018601,4.443679,22.425078,0.000000,0.0,5.005514
CCCAGTTTCGTTTATC_LUNG_N34,CCCAGTTTCGTTTATC,LUNG,N34,479,6.173786,1058,6.965080,44.706994,60.113422,73.629490,100.000000,3.686200,34.971645,0.000000,0.0,7.672589
CCCATACCAAGTCTGT_LUNG_N34,CCCATACCAAGTCTGT,LUNG,N34,858,6.755769,2249,7.718685,41.974211,56.425078,68.786127,84.081814,5.735883,28.368164,0.088928,0.0,0.000106
CCCAATCGTTTGGGCC_LUNG_N34,CCCAATCGTTTGGGCC,LUNG,N34,790,6.673298,1843,7.519692,39.283776,54.856213,67.498644,84.264786,5.046120,33.803581,0.054259,0.0,0.001323


In [26]:
import seaborn as sns
import matplotlib.pyplot as plt
#value = "pct_counts_mt"
#value = "n_genes"
#value = 'pct_counts_in_top_20_genes'
value = "log1p_total_counts"

sns.set(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})

g = sns.FacetGrid(df, row="sample", hue="sample", aspect=15, height=0.5, palette="tab20")

g.map(sns.kdeplot, value, clip_on=False, fill=True, alpha=1, linewidth=1.5)
g.map(sns.kdeplot, value, clip_on=False, color="w", lw=2)

g.map(plt.axhline, y=0, lw=2, clip_on=False)

def label(x, color, label):
    ax = plt.gca()
    ax.text(0, .2, label, fontweight="bold", color=color,
            ha="left", va="center", transform=ax.transAxes)

g.map(label, value)

g.figure.subplots_adjust(hspace=-.6)

g.set_titles("")
g.set(yticks=[], ylabel="")
g.despine(bottom=True, left=True)

for ax in g.axes.flat:
    ax.axvline(x=df[value].median(), color='r', linestyle='-')


plt.show()

/scratch/sah2p/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/seaborn/axisgrid.py:123: UserWarning: Tight layout not applied. tight_layout cannot make Axes height small enough to accommodate all Axes decorations.
  self._figure.tight_layout(*args, **kwargs)
/scratch/sah2p/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/seaborn/axisgrid.py:123: UserWarning: Tight layout not applied. tight_layout cannot make Axes height small enough to accommodate all Axes decorations.
  self._figure.tight_layout(*args, **kwargs)
/scratch/sah2p/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/seaborn/axisgrid.py:123: UserWarning: Tight layout not applied. tight_layout cannot make Axes height small enough to accommodate all Axes decorations.
  self._figure.tight_layout(*args, **kwargs)
/scratch/sah2p/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/seaborn/axisgrid.py:123: UserWarning: Tight layout not applied. tight_layout cannot make Axes height small enough to accommo

In [11]:
import doubletdetection
from scipy.stats import median_abs_deviation as mad
import numpy as np

In [12]:
def mad_outlier(adata, metric, nmads, upper_only = False):
    M = adata.obs[metric]
    
    if not upper_only:
        return (M < np.median(M) - nmads * mad(M)) | (M > np.median(M) + nmads * mad(M))
    
    return (M > np.median(M) + nmads * mad(M))

In [13]:
 clf = doubletdetection.BoostClassifier(
    n_iters=10,
    clustering_algorithm="louvain",
    standard_scaling=True,
    pseudocount=0.1,
    n_jobs=-1)

In [18]:
def pp(adata):
    adata = adata[adata.obs.pct_counts_mt < 10] #you can lower this based on the overal distribution of your dataset
    
    bool_vector = mad_outlier(adata, 'log1p_total_counts', 5) +\
            mad_outlier(adata, 'log1p_n_genes_by_counts', 5) +\
            mad_outlier(adata, 'pct_counts_in_top_50_genes', 5) +\
            mad_outlier(adata, 'pct_counts_mt', 3, upper_only = True)
    adata = adata[~bool_vector]

    adata.uns['cells_removed'] = sum(bool_vector)

    doublets = clf.fit(adata.X).predict(p_thresh=1e-3, voter_thresh=0.5)
    doublet_score = clf.doublet_score()

    adata.obs["doublet"] = doublets
    adata.obs["doublet_score"] = doublet_score

    adata.uns['doublets_removed'] = adata.obs.doublet.sum()
    adata = adata[adata.obs.doublet == 0]

    return adata

In [19]:
adata_by_sample = [pp(x) for x in adata_by_sample]

/tmp/ipykernel_2486374/2690890389.py:10: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns['cells_removed'] = sum(bool_vector)


  0%|          | 0/10 [00:00<?, ?it/s]

/tmp/ipykernel_2486374/2690890389.py:10: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns['cells_removed'] = sum(bool_vector)


  0%|          | 0/10 [00:00<?, ?it/s]

/tmp/ipykernel_2486374/2690890389.py:10: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns['cells_removed'] = sum(bool_vector)


...

  0%|          | 0/10 [00:00<?, ?it/s]

/tmp/ipykernel_2486374/2690890389.py:10: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns['cells_removed'] = sum(bool_vector)


  0%|          | 0/10 [00:00<?, ?it/s]

/tmp/ipykernel_2486374/2690890389.py:10: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns['cells_removed'] = sum(bool_vector)


  0%|          | 0/10 [00:00<?, ?it/s]

In [20]:
for adata in adata_by_sample:
    print(len(adata), adata.uns['cells_removed'], adata.uns['doublets_removed'])

2641 119 127.0
2756 185 95.0
2854 143 124.0
3889 153 81.0
1696 208 47.0
1243 148 25.0
2816 333 23.0
973 93 9.0
2218 212 113.0
2030 61 58.0
2056 87 64.0


In [21]:
df2 = pd.concat([x.obs for x in adata_by_sample])
df2 = df2.sort_values('sample')

In [22]:
df2

,barcode,Tissue,sample,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes,pct_counts_mt,pct_counts_ribo,pct_counts_hb,doublet,doublet_score
ACTATCTAGACAGAGA_LUNG_N01,ACTATCTAGACAGAGA,LUNG,N01,1254,7.134891,3020,8.013343,36.754967,48.874172,60.033113,75.033113,5.794702,21.622517,0.000000,0.0,0.000015
CCGGGATCACCTCGGA_LUNG_N01,CCGGGATCACCTCGGA,LUNG,N01,986,6.894670,2406,7.786136,37.032419,50.083126,62.967581,79.800499,5.320033,25.768911,0.000000,0.0,0.055152
CCGGGATCAGCTCGAC_LUNG_N01,CCGGGATCAGCTCGAC,LUNG,N01,1326,7.190676,3362,8.120589,37.477692,50.297442,61.511005,75.431291,4.878049,26.650803,0.000000,0.0,3.322630
CCGGTAGAGCACAGGT_LUNG_N01,CCGGTAGAGCACAGGT,LUNG,N01,547,6.306275,1198,7.089243,42.988314,59.432387,71.035058,96.076795,7.262104,38.397329,0.000000,0.0,2.092028
CCGGTAGAGCTAGGCA_LUNG_N01,CCGGTAGAGCTAGGCA,LUNG,N01,1204,7.094235,2815,7.943073,34.103020,47.069272,59.005329,74.991119,3.161634,22.202487,0.000000,0.0,0.209435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CCCAGTTTCAACGGCC_LUNG_N34,CCCAGTTTCAACGGCC,LUNG,N34,1022,6.930495,2903,7.973844,39.614192,53.806407,67.378574,82.018601,4.443679,22.425078,0.000000,0.0,5.005514
CCCAGTTTCGTTTATC_LUNG_N34,CCCAGTTTCGTTTATC,LUNG,N34,479,6.173786,1058,6.965080,44.706994,60.113422,73.629490,100.000000,3.686200,34.971645,0.000000,0.0,7.672589
CCCATACCAAGTCTGT_LUNG_N34,CCCATACCAAGTCTGT,LUNG,N34,858,6.755769,2249,7.718685,41.974211,56.425078,68.786127,84.081814,5.735883,28.368164,0.088928,0.0,0.000106
CCCAATCGTTTGGGCC_LUNG_N34,CCCAATCGTTTGGGCC,LUNG,N34,790,6.673298,1843,7.519692,39.283776,54.856213,67.498644,84.264786,5.046120,33.803581,0.054259,0.0,0.001323


In [27]:
#value = "pct_counts_mt"
#value = "n_genes"
# value = 'pct_counts_in_top_50_genes'
value = "log1p_total_counts"

sns.set(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})

g = sns.FacetGrid(df2, row="sample", hue="sample", aspect=15, height=2, palette="tab20")

g.map(sns.kdeplot, value, clip_on=False, fill=True, alpha=1, linewidth=1.5)
g.map(sns.kdeplot, value, clip_on=False, color="w", lw=2)

g.map(plt.axhline, y=0, lw=2, clip_on=False)

def label(x, color, label):
    ax = plt.gca()
    ax.text(0, .2, label, fontweight="bold", color=color,
            ha="left", va="center", transform=ax.transAxes)

g.map(label, value)

# g.figure.subplots_adjust(hspace=-.6)

g.set_titles("")
g.set(yticks=[], ylabel="")
g.despine(bottom=True, left=True)
g.figure.subplots_adjust(hspace=0.8)

for ax in g.axes.flat:
    ax.axvline(x=df2[value].median(), color='r', linestyle='-')


plt.show()

Continue from here:https://github.com/mousepixels/sanbomics_scripts/blob/main/sc2024/preprocessing.ipynb

In [29]:
df_cell_annotation = pd.read_csv("/scratch/sah2p/projects/G2PDeep-sc-LLM/data/GSE131907/GSE131907_Lung_Cancer_cell_annotation.txt",sep='\t')

In [30]:
df_cell_annotation[df_cell_annotation['Sample_Origin'] == 'nLung']

,Index,Barcode,Sample,Sample_Origin,Cell_type,Cell_type.refined,Cell_subtype
0,AAACCTGCAAGGTGTG_LUNG_N01,AAACCTGCAAGGTGTG,LUNG_N01,nLung,Myeloid cells,Myeloid cells,mo-Mac
1,AACTCCCGTTCACCTC_LUNG_N01,AACTCCCGTTCACCTC,LUNG_N01,nLung,Myeloid cells,Myeloid cells,mo-Mac
2,AACTCCCTCACGCGGT_LUNG_N01,AACTCCCTCACGCGGT,LUNG_N01,nLung,Myeloid cells,Myeloid cells,mo-Mac
3,AAGGAGCGTGTGCGTC_LUNG_N01,AAGGAGCGTGTGCGTC,LUNG_N01,nLung,Myeloid cells,Myeloid cells,mo-Mac
4,AAGGTTCAGGTACTCT_LUNG_N01,AAGGTTCAGGTACTCT,LUNG_N01,nLung,Myeloid cells,Myeloid cells,mo-Mac
...,...,...,...,...,...,...,...
42990,TTTGCGCCATTCTCAT_LUNG_N34,TTTGCGCCATTCTCAT,LUNG_N34,nLung,T lymphocytes,NaN,NaN
42991,TTTGGTTCAATAGCAA_LUNG_N34,TTTGGTTCAATAGCAA,LUNG_N34,nLung,T lymphocytes,NaN,NaN
42992,TTTGGTTTCAGTGTTG_LUNG_N34,TTTGGTTTCAGTGTTG,LUNG_N34,nLung,T lymphocytes,NaN,NaN
42993,TTTGTCAAGATGTCGG_LUNG_N34,TTTGTCAAGATGTCGG,LUNG_N34,nLung,T lymphocytes,NaN,NaN


In [31]:
df_annotated = pd.merge(df_cell_annotation[df_cell_annotation['Sample_Origin'] == 'nLung'][['Barcode','Cell_type']],df2, left_on='Barcode',right_on='barcode')

In [38]:
# df_annotated.drop_duplicates(inplace=True)
combined_data.obs = df_annotated

NameError: name 'combined_data' is not defined

In [33]:
df2.nunique()

barcode                        24763
Tissue                             1
sample                            11
n_genes_by_counts               2142
log1p_n_genes_by_counts         2142
total_counts                    6364
log1p_total_counts              6364
pct_counts_in_top_50_genes     24199
pct_counts_in_top_100_genes    24380
pct_counts_in_top_200_genes    24391
pct_counts_in_top_500_genes    24239
pct_counts_mt                  22906
pct_counts_ribo                24586
pct_counts_hb                   5581
doublet                            1
doublet_score                  12058
dtype: int64

In [34]:
df_nlung_ann = df_cell_annotation[df_cell_annotation['Sample_Origin'] == 'nLung']
df_nlung_ann = df_nlung_ann[['Barcode','Cell_type']].drop_duplicates(subset=['Barcode'])
# df_nlung_ann.set_index('Barcode',inplace=True)


In [35]:
df_nlung_ann[df_nlung_ann['Barcode']=='AAACCTGGTACCAGTT']

,Barcode,Cell_type
27629,AAACCTGGTACCAGTT,T lymphocytes


In [28]:

# Combine the list of AnnData objects into one
combined_adata = adata_by_sample[0].concatenate(adata_by_sample[1:])

# Check the resulting combined AnnData object
print(combined_adata)

/tmp/ipykernel_2486374/267019385.py:2: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  combined_adata = adata_by_sample[0].concatenate(adata_by_sample[1:])


AnnData object with n_obs × n_vars = 25172 × 29634
    obs: 'barcode', 'Tissue', 'sample', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'pct_counts_mt', 'pct_counts_ribo', 'pct_counts_hb', 'doublet', 'doublet_score', 'batch'
    var: 'mt', 'ribo', 'hb', 'n_cells_by_counts-0', 'mean_counts-0', 'log1p_mean_counts-0', 'pct_dropout_by_counts-0', 'total_counts-0', 'log1p_total_counts-0', 'n_cells_by_counts-1', 'mean_counts-1', 'log1p_mean_counts-1', 'pct_dropout_by_counts-1', 'total_counts-1', 'log1p_total_counts-1', 'n_cells_by_counts-10', 'mean_counts-10', 'log1p_mean_counts-10', 'pct_dropout_by_counts-10', 'total_counts-10', 'log1p_total_counts-10', 'n_cells_by_counts-2', 'mean_counts-2', 'log1p_mean_counts-2', 'pct_dropout_by_counts-2', 'total_counts-2', 'log1p_total_counts-2', 'n_cells_by_counts-3', 'mean_counts-3', 'log1p_m

In [36]:
combined_adata.obs['cell_type'] = combined_adata.obs_names.map(df_nlung_ann['Cell_type'])

# Step 4: Verify the annotations were added successfully
# print(combined_adata.obs[['cell_type']].head())


In [153]:
df = pd.merge(combined_adata.obs, df_nlung_ann, right_on='Barcode',left_on='barcode')

In [154]:
combined_adata.obs = df

In [156]:
combined_adata.var_names

Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2ML1-AS1',
       'A2ML1-AS2', 'A4GALT', 'A4GNT',
       ...
       'ZXDA', 'ZXDB', 'ZXDC', 'ZYG11A', 'ZYG11B', 'ZYX', 'ZZEF1', 'ZZZ3',
       'bP-21264C1.2', 'bP-2189O9.3'],
      dtype='object', length=29634)

In [138]:
combined_adata.obs['barcode'] =combined_adata.obs['barcode'].str.strip()

In [143]:
combined_adata.obs[combined_adata.obs['barcode'] =='AAACCTGGTACCAGTT']

,barcode,Tissue,sample,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes,pct_counts_mt,pct_counts_ribo,pct_counts_hb,doublet,doublet_score,batch,cell_type
AAACCTGGTACCAGTT_LUNG_N28-0,AAACCTGGTACCAGTT,LUNG,N28,860,6.758095,2065,7.63337,43.535109,55.496368,67.409201,82.566586,4.794189,29.491525,0.048426,0.0,0.000746,0,NaN


In [157]:
# Optionally save the combined AnnData object
combined_adata.write('/scratch/sah2p/projects/G2PDeep-sc-LLM/data/GSE131907/combined_adata.h5ad')

In [159]:
combined_adata.obs.columns

Index(['barcode', 'Tissue', 'sample', 'n_genes_by_counts',
       'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts',
       'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes',
       'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes',
       'pct_counts_mt', 'pct_counts_ribo', 'pct_counts_hb', 'doublet',
       'doublet_score', 'batch', 'cell_type', 'Barcode', 'Cell_type'],
      dtype='object')

In [165]:
combined_adata.var = combined_adata.var.reset_index()

In [167]:
combined_adata.var.columns = ['gene_name', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts',
       'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts',
       'log1p_total_counts']

In [170]:
combined_adata.write('/scratch/sah2p/projects/G2PDeep-sc-LLM/data/GSE131907/combined_adata.h5ad')

In [177]:
import scanpy as sc

# Perform preprocessing steps (if not already done)
sc.pp.normalize_total(combined_adata, target_sum=1e4)
sc.pp.log1p(combined_adata)
sc.pp.highly_variable_genes(combined_adata)


# Ensure that neighbors have been computed before running UMAP
sc.pp.pca(combined_adata)                # Perform PCA first (if not already done)
sc.pp.neighbors(combined_adata)          # Compute the nearest neighbors graph

# Now, compute UMAP
sc.tl.umap(combined_adata)

# After running UMAP, it should store the result in adata.obsm['X_umap']
print(combined_adata.obsm['X_umap'])

[[ 3.3106172   5.213119  ]
 [-4.3502393  -0.1170982 ]
 [ 7.057697    8.15754   ]
 ...
 [ 5.673259    9.745809  ]
 [-6.200548    0.13844946]
 [ 5.5718994   7.5935655 ]]


In [178]:
sc.pl.umap(combined_adata, color='Cell_type', save="testing_plot.png")